In [47]:
import  requests
import  pandas as pd
import json

In [48]:
url="https://city.imd.gov.in/yogi/php/chart_min_max.php"
payload={
"INDEX_NUMBER":43063,
"DATE1":"2024-05-29",
"DATE2":"2024-10-05"
}

headers = {
    'Content-Type': 'application/x-www-form-urlencoded',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
    'Connection': 'keep-alive',

}

response = requests.post(url, data=payload, headers=headers, verify=False)
response.encoding = 'utf-8'
# Check the status code


C:\Users\shind\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'city.imd.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [49]:
if response.status_code == 200:
    print('Request successful!')
    print(type(response.text))
    data = json.loads(response.text)
    filt_data=data['data_max'][1:]
    #print(filt_data)
    df1 = pd.json_normalize(filt_data)
    #print(data['data_max'])
    
    filt_data2=data['data_min'][1:]
    #print(filt_data)
    df2 = pd.json_normalize(filt_data2)
    #print(df1.shape)
    print (df1, df2)
else:
    print(f'Request failed with status code: {response.status_code}')

Request successful!
<class 'str'>
        Datee Parameter1 Parameter2
0    29-05-24       35.7       36.3
1    30-05-24       36.5       36.3
2    31-05-24       37.1       35.7
3    01-06-24       37.2       35.7
4    02-06-24       36.6       35.7
..        ...        ...        ...
125  01-10-24       34.9       30.9
126  02-10-24       33.8       30.9
127  03-10-24       31.9       31.3
128  04-10-24       31.9       31.3
129  05-10-24       None       31.3

[130 rows x 3 columns]         Datee Parameter1 Parameter2
0    29-05-24       24.1       23.8
1    30-05-24       24.4       23.8
2    31-05-24       24.9       23.6
3    01-06-24       24.9       23.6
4    02-06-24       25.1       23.6
..        ...        ...        ...
125  01-10-24       21.9       21.0
126  02-10-24       23.4       21.0
127  03-10-24       23.2       20.6
128  04-10-24       21.5       20.6
129  05-10-24       20.2       20.6

[130 rows x 3 columns]


In [50]:
df1.rename(columns={'Datee': 'date', 'Parameter1': 'Tmax','Parameter2': 'Normal of Tmax'}, inplace=True)
df2.rename(columns={'Datee': 'date', 'Parameter1': 'Tmin','Parameter2': 'Normal of Tmin'}, inplace=True)

In [51]:
df1['date'] = pd.to_datetime(df1['date'], format='mixed')
df2['date'] = pd.to_datetime(df2['date'], format='mixed')

In [52]:
merged_df = pd.merge(df1, df2, on='date', how='inner')
merged_df

,date,Tmax,Normal of Tmax,Tmin,Normal of Tmin
0,2024-05-29,35.7,36.3,24.1,23.8
1,2024-05-30,36.5,36.3,24.4,23.8
2,2024-05-31,37.1,35.7,24.9,23.6
3,2024-01-06,37.2,35.7,24.9,23.6
4,2024-02-06,36.6,35.7,25.1,23.6
...,...,...,...,...,...
125,2024-01-10,34.9,30.9,21.9,21.0
126,2024-02-10,33.8,30.9,23.4,21.0
127,2024-03-10,31.9,31.3,23.2,20.6
128,2024-04-10,31.9,31.3,21.5,20.6


In [55]:
#null data printing
null_mask = merged_df.isnull()
print("\nNull Mask:")
#print(null_mask)
print(merged_df[merged_df.isnull().any(axis=1)])


Null Mask:
          date  Tmax Normal of Tmax  Tmin Normal of Tmin
25  2024-06-23  None           30.4  23.6           23.0
79  2024-08-16  None           27.6  22.2           21.8
129 2024-05-10  None           31.3  20.2           20.6


In [56]:
merged_df_cleaned = merged_df.dropna()
merged_df_cleaned

,date,Tmax,Normal of Tmax,Tmin,Normal of Tmin
0,2024-05-29,35.7,36.3,24.1,23.8
1,2024-05-30,36.5,36.3,24.4,23.8
2,2024-05-31,37.1,35.7,24.9,23.6
3,2024-01-06,37.2,35.7,24.9,23.6
4,2024-02-06,36.6,35.7,25.1,23.6
...,...,...,...,...,...
124,2024-09-30,33.2,30.9,20.4,21.0
125,2024-01-10,34.9,30.9,21.9,21.0
126,2024-02-10,33.8,30.9,23.4,21.0
127,2024-03-10,31.9,31.3,23.2,20.6


In [66]:
merged_df_cleaned.to_csv('pune-shivajinagar-{}-to-{}.csv'.format(payload['DATE1'],payload['DATE2']))

In [67]:
#reading as test
new_fd=pd.read_csv("./pune-shivajinagar-2024-05-29-to-2024-10-05.csv")

In [65]:
new_fd

,Unnamed: 0,date,Tmax,Normal of Tmax,Tmin,Normal of Tmin
0,0,2024-05-29,35.7,36.3,24.1,23.8
1,1,2024-05-30,36.5,36.3,24.4,23.8
2,2,2024-05-31,37.1,35.7,24.9,23.6
3,3,2024-01-06,37.2,35.7,24.9,23.6
4,4,2024-02-06,36.6,35.7,25.1,23.6
...,...,...,...,...,...,...
122,124,2024-09-30,33.2,30.9,20.4,21.0
123,125,2024-01-10,34.9,30.9,21.9,21.0
124,126,2024-02-10,33.8,30.9,23.4,21.0
125,127,2024-03-10,31.9,31.3,23.2,20.6
